## 밑바닥부터 시작하는 딥러닝2
- 2장. 자연어와 단어의 분산 표현

In [5]:
text = 'You say goodbye and I say hello.'

text = text.lower()
text = text.replace('.',' .')
text

'you say goodbye and i say hello .'

In [6]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [7]:
# 단어를 {id : 단어} 딕셔너리 형태로 변환(검색 용이)
word_to_id ={}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [8]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [9]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [10]:
id_to_word[1]

'say'

In [11]:
word_to_id['you']

0

In [12]:
# 단어 목록(word_to_id)을 단어ID 목록으로 변환(현재 key가 단어이므로 숫자로 변경)

import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [13]:
# 전처리 함수 정의(위치 : common/util.py)

def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    
    word_to_id ={}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [14]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
corpus, word_to_id, id_to_word 

(array([0, 1, 2, 3, 4, 1, 5, 6]),
 {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6},
 {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'})

- 동시발생 행렬(주변에 어떤 단어가 몇 번이나 등장하는지 집계)

In [16]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)
print()
print(id_to_word)

[0 1 2 3 4 1 5 6]

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [17]:
#동시발생 행렬(수동 작성)

C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0]
], dtype=np.int32)

print(C[0]) #ID=0인 단어의 벡터
print()
print(C[4]) #ID=4인 단어의 벡터
print()
print(C[word_to_id['goodbye']]) #'goodbye'의 벡터

[0 1 0 0 0 0 0]

[0 1 0 1 0 0 0]

[0 1 0 1 0 0 0]


In [18]:
# 동시발생 행렬 자동 생성 함수 정의(위치 : common/util.py)

def create_co_matrix(corpus, vocab_size, window_size=1):
    '''동시발생 행렬 생성

    :param corpus: 말뭉치(단어 ID 목록)
    :param vocab_size: 어휘 수
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return: 동시발생 행렬
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1 #동시 등장 단어에 +1 추가

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

# 코사인 유사도 함수 정의(위치 : common/util.py)

def cos_similarity(x, y, eps=1e-8):
    '''코사인 유사도 산출

    :param x: 벡터
    :param y: 벡터
    :param eps: '0으로 나누기'를 방지하기 위한 작은 값
    :return:
    '''
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [19]:
# 유사도 구하기

import sys
sys.path.append('..')
from common.util import preprocess,create_co_matrix,cos_similarity

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]

print(cos_similarity(c0,c1))

0.7071067691154799


- 유사 단어의 랭킹 표시

In [20]:
# 유사 단어 랭킹 함수 정의(위치 : common/util.py)

def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):

    # 검색어 꺼내기
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' %query)
        return
    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    #코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    #코사인 유사도 기준 내림차순 출력
    count = 0
    for i in (-1 * similarity).argsort(): #음수화하고 오름차순 정렬 -> 내림차순 효과
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [21]:
x = np.array([100,-20,2])
x.argsort()

array([1, 2, 0], dtype=int64)

In [22]:
# 랭킹 구하기

import sys
sys.path.append('..')
from common.util import preprocess,create_co_matrix,most_similar

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id,id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
